In [109]:
import pandas as pds
import json
from rdflib import Graph, ConjunctiveGraph, RDFS, RDF, OWL, URIRef, Literal
from hashlib import md5
from pandasql import sqldf

def pysqldf(q):
    return sqldf(q, globals())

## Build dataframe containing enviromental level data values

In [2]:
all_data_df = pds.read_csv("data/Biosample_all.tsv.gz", sep="\t")

/Users/wdduncan/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (13,14,16,18,19,23,26,27,28,29,30,34,35,36,37,39,41,42,43,44,45,52,57,62,70,72,73,76,77,78,79,86,98,99,100,103,104,105,106,111,119,124,133,134,135,142,144,146,149,165,191,208,229,233,252,253,254) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Normalize column names to lower case and trim/strip spaces

In [3]:
clean_columns = [c.lower().strip() for c in all_data_df.columns]
# clean_columns

In [4]:
all_data_df.columns = clean_columns

## Create subset that contains just the GOLD elevels data

In [5]:
gold_elevels = ['ecosystem', 'ecosystem_category', 'ecosystem_type', 'ecosystem_subtype', 'specific_ecosystem']
gold_elevelsdf = all_data_df[gold_elevels]

### Clean data: 
* replace with nan with empyt string
* make all values lowercase 
* trim spaces

In [6]:
gold_elevelsdf.fillna("", inplace=True)
gold_elevelsdf = gold_elevelsdf.applymap(lambda x: x.lower().strip())    
gold_elevelsdf.head()

/Users/wdduncan/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4244: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  **kwargs


,ecosystem,ecosystem_category,ecosystem_type,ecosystem_subtype,specific_ecosystem
0,host-associated,plants,phyllosphere,caulosphere,
1,engineered,food production,dairy products,,
2,engineered,food production,dairy products,,
3,engineered,food production,dairy products,,
4,engineered,food production,dairy products,,


## Build dataframe of unique label paths and their checksums
#### For example: 'enviromental > aquatic > freshwater > sediment'

In [7]:
## helper functions for creating label path, hash, and iri
def make_label_path(row, include_missing=False):
    path_list = list(row)
    temp = [e for e in path_list if e != ""] # remove all empty strings
    if len(temp) > 0:
        if include_missing:
            ## if the value in the list an empty string (e.g., ['host-associated', 'plants', 'endosphere', ''])
            ## this will put an " > " it (e.g., host-associated > plants > endosphere >)
            return " > ".join(path_list)
        else:
            ## this only retuns a path devoid of empty string
            ## e.g, ['host-associated', 'plants', 'endosphere', ''] returns host-associated > plants > endosphere
            return " > ".join(temp)
    else:
        return ""


def make_parent_label_path(label_path):
    path_list = label_path.split(" > ")
    if len(path_list) > 1:
        temp = path_list[0:-1]
        return " > ".join(temp)
    else:
        return ""


def make_row_list(row, subset_list=[]):
    if len(subset_list) > 0:  # only get values in subset list
        row_list = [str(v) for k,v in row.to_dict().items() 
                           if k in subset_list]
    else:
        row_list = [str(e) for e in list(row)]
        
    return row_list


def make_row_hash(row, subset_list=[]):
    row_list = make_row_list(row, subset_list)
    
    temp = ["" if pds.isnull(e) else e for e in row_list] # replace NaNs with ''
    temp = "".join(temp)
#     print("row: ", temp)
    if "host-associatedmicrobialbacteriaunclassifiedunclassified" == temp:
        print(make_hash(temp))
    return make_hash(temp)


def make_parent_hash(row, subset_list=[]):
    row_list = make_row_list(row, subset_list)
    row_list = row_list[0:-1] # to get the parent, exclude last value in list

    temp = ["" if pds.isnull(e) else e for e in row_list] # replace NaNs with ''
    temp = "".join(temp)
#     print("parent: ", temp)
    return make_hash(temp)


def make_hash(val):
    if len(val) > 0:
        hash = md5(val.encode('utf-8'))
        return str(hash.hexdigest())
    else:
        return ""


def make_iri(val, prefix="http://purl.obolibrary.org/obo/GOLD_"):
    if None != val and len(val) > 0:
#         hash = make_hash(val)
        return f"{prefix}{val}"
    else:
        return ""


def make_annotation_dict(value_list, prefix="http://purl.obolibrary.org/obo/GOLD_"):
    annotation_dict = {}
    for val in value_list:
        val = str(val).lower().strip()
        annotation_dict[val] = \
            {'iri': make_iri(str(val), prefix=prefix), 'label': val}
    return annotation_dict


def make_json_annotation(row, annotation_dict):
    annotation_value_dict = {}
    annotation_dict_keys = annotation_dict.keys()
    for k,v in row.to_dict().items():
        k = str(k).lower().strip()
        if v != "" and (k in annotation_dict_keys):
            annotation_value_dict[annotation_dict[k]['iri']] = v
    return json.dumps(annotation_value_dict)

In [80]:
ontdf = pds.DataFrame(columns=['label_path', 'parent_label_path', 'iri', 'parent_iri', 'annotation'])
annotation_dict = make_annotation_dict(gold_elevels)

#gold_elevelsdf = gold_elevelsdf.head() # used for testing

for i in range(len(gold_elevels)):
    df = gold_elevelsdf[gold_elevels[0:i + 1]].fillna("").drop_duplicates()
    df['label_path'] = df.apply(lambda row: make_label_path(row), axis=1)
    df['row_hash'] = df.apply(lambda row: make_row_hash(row, subset_list=gold_elevels), axis=1)
    
    df['parent_label_path'] = df.apply(lambda row: make_parent_label_path(row['label_path']), axis=1)
    df['parent_hash'] = df.apply(lambda row: make_parent_hash(row, subset_list=gold_elevels), axis=1)
    
    df['iri'] = df.apply(lambda row: make_iri(row['row_hash']), axis=1)
    df['parent_iri'] = df.apply(lambda row: make_iri(row['parent_hash']), axis=1)
    
    df['annotation'] = df.apply(lambda row: make_json_annotation(row, annotation_dict), axis=1)
    ontdf = ontdf.append(df[['label_path', 'parent_label_path', 'iri', 'parent_iri', 'annotation']]).drop_duplicates()

8933c51848ce54db789cba16bf6707ea


In [78]:
## visually examine output
# pds.set_option('max_rows', None)
# pds.set_option('display.max_colwidth', 1000)
# print(ontdf[['label_path', 'parent_label_path']].head())
# print(ontdf[['iri', 'parent_iri']].head())
# print(ontdf[['label_path', 'parent_label_path', 'iri', 'parent_iri']].head())
# print(ontdf[['annotation']].head())
# print(len(ontdf))
# ontdf[['label_path', 'parent_label_path', 'iri', 'parent_iri']].head()

In [10]:
## check that number of unique iris matches the number of rows
print("Nuumber of rows:", len(ontdf))
print("Number of iris: ", len(ontdf.iri.unique()))

Nuumber of rows: 1073
Number of iris:  833


In [11]:
## check the number of unique vs non-unique parent iris
## there should be less unique ones; there is a one-to-many relation between parent iris and iris(i.e., parents have many children)
print("Number of unique parent iris: ", len(ontdf.parent_iri.unique()))
print("Number of parent iris: ", len(ontdf.parent_iri))

Number of unique parent iris:  480
Number of parent iris:  1073


In [12]:
## check that the number of non-empty lable paths matches the number of non-empty iris
## there should be one less (see code below which returns one row of empty strings)
print("Number of non-empty iris: ", len(ontdf[ontdf.iri != ''].iri.unique()))
print("Number of non-empty label paths: ", len(ontdf[ontdf.label_path != ''].label_path.unique()))

print("Number of non-empty parent iris: ", len(ontdf[ontdf.parent_iri != ''].parent_iri.unique()))
print("Number of non-empty parent label paths: ", len(ontdf[ontdf.parent_label_path != ''].parent_label_path.unique()))

Number of non-empty iris:  832
Number of non-empty label paths:  832
Number of non-empty parent iris:  479
Number of non-empty parent label paths:  399


In [13]:
## only only records (show above) should have the same parent iri and iri value
print("Number of matching iri/parent iri: ", len(ontdf[ontdf.iri == ontdf.parent_iri]))
# ontdf[ontdf.iri == ontdf.parent_iri]

Number of matching iri/parent iri:  241


In [14]:
## examine those records w/o a parent iri
ontdf[ontdf.parent_iri == '']

,label_path,parent_label_path,iri,parent_iri,annotation
0,host-associated,,http://purl.obolibrary.org/obo/GOLD_daf7fb2e82...,,"{""http://purl.obolibrary.org/obo/GOLD_ecosyste..."
1,engineered,,http://purl.obolibrary.org/obo/GOLD_fee6b5a458...,,"{""http://purl.obolibrary.org/obo/GOLD_ecosyste..."
7,environmental,,http://purl.obolibrary.org/obo/GOLD_3bbfdace1e...,,"{""http://purl.obolibrary.org/obo/GOLD_ecosyste..."
8786,,,,,{}


## Use rdflib to build ontology from dataframe

In [15]:
g = Graph() # instantiate graph

### Create annotaiton properties for each of the columns (saved in the annotation dictionary)

In [16]:
for k, v in annotation_dict.items():
    ## note: the value is a dict with keys 'iri' and 'label'
    g.add((URIRef(v['iri']), RDF.type, OWL.AnnotationProperty))
    g.add((URIRef(v['iri']), RDFS.label, Literal(v['label'])))

### Add each row from the ontology dataframe to graph

In [17]:
for (ix, label_path, parent_label_path, iri, parent_iri, annotation) in ontdf.itertuples(): # ontdf.head(100).itertuples():
    if len(iri) > 0:
        ## add iri to graph
        g.add((URIRef(iri), RDF.type, OWL.Class))
        g.add((URIRef(iri), RDFS.label, Literal(label_path)))
        
        ## add iri annotations to graph (note: annotation is a json string of form iri:value)
        ann = json.loads(annotation)
        for k, v in ann.items():
            g.add((URIRef(iri), URIRef(k), Literal(v)))
    
    if len(parent_iri) > 0:
        g.add((URIRef(iri), RDFS.subClassOf, URIRef(parent_iri))) # add parent iri to graph

In [18]:
## save graph (note: different formatats (e.g., turtle) are possible)
g.serialize(destination='output/gold-dataset-translation.owl', format='xml')

In [128]:
for r in g.query(query): print(r)

(rdflib.term.Literal('832', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')),)


## Translate data into JSON-LD

### Build JSON-LD context

In [87]:
context_dict = {"id": "@id", 
                "type": "@type",
                "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
                "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
                "owl": "http://www.w3.org/2002/07/owl#",
                "label": "rdfs:label",
                "parent": "rdfs:subClassOf"}
for k, v in annotation_dict.items():
    ## note: the value is a dict with keys 'iri' and 'label'
    context_dict[v['label']] = v['iri']
context_dict

{'id': '@id',
 'type': '@type',
 'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#',
 'rdfs': 'http://www.w3.org/2000/01/rdf-schema#',
 'owl': 'http://www.w3.org/2002/07/owl#',
 'label': 'rdfs:label',
 'parent': 'rdfs:subClassOf',
 'ecosystem': 'http://purl.obolibrary.org/obo/GOLD_ecosystem',
 'ecosystem_category': 'http://purl.obolibrary.org/obo/GOLD_ecosystem_category',
 'ecosystem_type': 'http://purl.obolibrary.org/obo/GOLD_ecosystem_type',
 'ecosystem_subtype': 'http://purl.obolibrary.org/obo/GOLD_ecosystem_subtype',
 'specific_ecosystem': 'http://purl.obolibrary.org/obo/GOLD_specific_ecosystem'}

In [134]:
context_str = '"@context": ' + json.dumps(context_dict, indent=4)

### Build class hierarchy in JSON-LD

In [135]:
data_list = []
for (ix, label_path, parent_label_path, iri, parent_iri, annotation) in ontdf.itertuples(): # ontdf.head(100).itertuples():
    item_dict = {}
    if len(iri) > 0:
        ## add iri to graph
        item_dict["id"] = iri
        item_dict["type"] = "owl:Class"
        item_dict["label"] = label_path
        
        ## add iri annotations to graph (note: annotation is a json string of form iri:value)
        ann = json.loads(annotation)
        for ann_key, ann_val in ann.items():
            for context_key, context_val in context_dict.items():
                if context_val == ann_key: item_dict[context_key] = ann_val
    
    if len(parent_iri) > 0:
        item_dict["parent"] = parent_iri
    
    if len(item_dict) > 0:
        data_list.append(json.dumps(item_dict, indent=4))

In [136]:
# data_list

In [137]:
data_str = '"@graph": [' + ", \n ".join(data_list) +"]"

In [144]:
doc_str = "{\n" + context_str + ", \n" + data_str + "\n}"

In [145]:
# doc_str

In [146]:
gc = ConjunctiveGraph()
gc.parse(data=doc_str, format="json-ld")

<Graph identifier=Na5c5435c6255489fbf03b5883f6c4744 (<class 'rdflib.graph.Graph'>)>

In [147]:
# gc.serialize(destination='output/temp.owl', format='json-ld')

In [148]:
query = """
prefix owl: <http://www.w3.org/2002/07/owl#>

select (count(?c) as ?count) 
where {?c a owl:Class}
"""
for r in gc.query(query): print(r)

(rdflib.term.Literal('832', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')),)


In [149]:
with open("output/temp.txt", "w") as f:
#     f.write(json.dumps(doc_str, indent=4))
    f.write(doc_str)